In [1]:
!pip install pypdf
!pip install pymupdf
!pip install google-search-results
!pip install serpapi
!pip install groq
!pip install langchain_community langchain_text_splitters
!pip install sentence-transformers
!pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 54.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32010 sha256=4c4e10aa1b8467c47564febafda98e0db36e889c193e5a759f1dbe1067110c9a
  Stored in directory: /root/.cache/pip/wheels/d3/b2/c3/03302d12bb44a2cdff3c9371f31b72c0c4e84b8d2285eeac53
Successfully built google-search-results
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.8 MB/s eta 0:00:00
 

In [2]:
from pypdf import PdfReader
import pymupdf
import re
import os
import requests
from serpapi.google_scholar_search import GoogleScholarSearch
from groq import Groq
import time
from pypdf import PdfReader, PdfWriter
from pypdf.annotations import Text
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings


Retrieve references from paper using named destinations

In [59]:
def refs_dict(pdf_path):
  cites = {}
  reader = PdfReader(pdf_path)
  doc = pymupdf.open(pdf_path)
  height = reader.pages[0].mediabox.height
  for cite, info in reader.named_destinations.items():
    if cite[:4] == 'cite':
        x1 = info['/Left']
        y1 = height-info['/Top']
        x2 = x1+400
        y2 = y1+20
        for i in range(len(reader.pages)):
          if reader.pages[i] == info['/Page']:
            page = doc[i]
            rect = pymupdf.Rect(x1,y1,x2,y2)
            cites[cite] = page.get_textbox(rect)
  return cites

Create folder to store source files

In [60]:
def make_folder(pdf_path):
  title = re.sub(r'\W+', '', pdf_path)[:-3]
  title = f"{title}_refs"
  if not os.path.exists(title):
      os.makedirs(title)
  return title

Retrieve documents via Serp API Google Scholar search

In [ ]:
def get_doc(folder, title, cite):
    api_key = os.environ.get("SERP_API_KEY")
    filename = re.sub(r'\W+', '', cite[5:])

    search = GoogleScholarSearch({"q": title, "api_key":api_key})
    data = search.get_dict()
    url = data['organic_results'][0]['resources'][0]['link']
    r = requests.get(url)

    with open(f"/content/{folder}/{filename}.pdf", "wb") as f:
      f.write(r.content)
    return f"{filename}.pdf"

def get_docs(folder, cites):
  for cite, title in cites.items():
     get_doc(folder, title, cite)


Chunk text to fit in ctx length + Groq rate limit

In [6]:
# HELPER FUNCS FOR summarize()

def texthalf(text):
  return [text[:len(text)//2],text[len(text)//2:]]

def text_extractor(file, chunks = 2):
    text = ""
    reader = PdfReader(file)
    for page in reader.pages:
      text+=page.extract_text()
    return re.sub('\n', ' ', text)

def get_chunked(folder, file):
    return texthalf(text_extractor(f"/content/{folder}/{file}"))

Call Groq Llama 3 70B to summarize the halves of the document then combine the two

In [51]:
def summarize(text):
    chat_completion = client.chat.completions.create(

      messages=[

          {
              "role": "system",
              "content": """You are a professional summarizer hired by me to provide context to citations within papers. You will be provided with the text
              within which the citation appears as well as relevant text chunks from the cited paper. Please use the text chunks to provide context for the citation.
              Make your explanation as brief as possible while still fully explaining the citation's relevance to the passage.
              """
          },
          {
              "role": "user",
              "content": text,
          }
      ],
      model="llama3-70b-8192",
      temperature=0.5,
      max_tokens=1024,
      top_p=1,
      stop=None,
      stream=False,
    )
    return chat_completion.choices[0].message.content

def final(texts, ctx):
  return f"""Explain the following passage:
"{texts}"
using this context from the paper cited: "{ctx}". CONTEXTUALIZE: """

def summarize_paper(text, ref_ctx):
  inputs = final(text, ref_ctx)
  return summarize(inputs)



Match citation summaries to citation locations,

In [72]:
def locs_dict(pdf_path, cites):
  reader = PdfReader(pdf_path)
  locs = {cite: {} for cite in cites}
  for page_num, page in enumerate(reader.pages):
      if '/Annots' in page:
          for annot in page['/Annots']:
              obj = annot.get_object()
              if '/Subtype' in obj and obj['/Subtype'] == '/Link' and '/A' in obj and obj['/A']['/S'] == '/GoTo':
                  cite_ref = obj['/A']['/D']
                  if cite_ref in locs.keys():
                    if not locs[cite_ref]:
                      locs[cite_ref] = {page_num:[obj['/Rect']]}
                    elif page_num not in locs[cite_ref].keys():
                      locs[cite_ref][page_num] = [obj['/Rect']]
                    else:
                      locs[cite_ref][page_num].append(obj['/Rect'])
  return locs

get surrounding context

In [78]:
def get_ctx(doc, reader, cite):
  ctx = {}
  for page_num, page in enumerate(reader.pages):
    if '/Annots' in page:
        for annot in page['/Annots']:
            obj = annot.get_object()
            if '/Subtype' in obj and obj['/Subtype'] == '/Link' and '/A' in obj and obj['/A']['/S'] == '/GoTo':
                cite_ref = obj['/A']['/D']
                if cite_ref in cite:
                  info = obj['/Rect']
                  x1 = 100
                  y1 = 792-info[1]-10
                  x2 = 500
                  y2 = y1+25
                  rect = pymupdf.Rect(x1,y1,x2,y2)
                  if page_num not in ctx.keys():
                    ctx[page_num] = [re.sub('\n', ' ', doc[page_num].get_textbox(rect))]
                  else:
                    ctx[page_num].append(re.sub('\n', ' ', doc[page_num].get_textbox(rect)))
  return ctx


def ctx_dict(pdf_path, cites):
  doc = pymupdf.open(pdf_path)
  reader = PdfReader(pdf_path)
  cite_dict = {}
  for cite in cites:
    cite_dict[cite] = get_ctx(doc, reader, cite)
  return cite_dict


In [54]:
def init_rag(folder):
    docs = []
    text_splitter = CharacterTextSplitter(chunk_size=400, chunk_overlap=50, separator = " ")
    for file in os.listdir(folder):
        text = text_extractor(folder+"/"+file)
        docs.append(text)
    texts = text_splitter.create_documents(docs)
    embeddings = HuggingFaceEmbeddings()
    db = FAISS.from_documents(texts, embeddings)
    retreiver = db.as_retriever()
    return retreiver

In [92]:
def summaries_dict(folder_path, ctx):
  client = os.environ.get("GROQ_API_KEY")
  summ = {}
  index = init_rag(folder_path)
  for cite, ctxs in ctx.items():
    summ[cite] = {}
    for page, ctx in ctxs.items():
      summ[cite][page] = []
      for c in ctx:
        docs = index.get_relevant_documents(c, k = 2)
        info = str([doc.page_content for doc in docs])
        summary = summarize_paper(c, info)
        print(summary)
        summ[cite][page].append(summary)
  return summ




Example:

In [49]:
def add_popup_annotation(writer, page_number, rect, content):
    annotation = Text(text = content, rect = rect)
    writer.add_annotation(page_number=page_number, annotation=annotation)

def annotations(p, c, l, s):
  reader = PdfReader(p)
  writer = PdfWriter()
  for page in reader.pages:
    writer.add_page(page)
  for cite in c.keys():
    print(c.keys())
    for page, coords in l[cite].items():
      for i in range(len(coords)):
        add_popup_annotation(writer, page, coords[i], re.sub('\n', '', s[cite][page][i]))
  with open("annotated.pdf", "wb") as fp:
      writer.write(fp)

dict_keys(['cite.DBLP:journals/corr/BritzGLL17'])


In [93]:
groq_key = os.getenv("GROQ_KEY")
serp_key = os.getenv("SERP_KEY")

pdf_path = "1706.03762v7.pdf"

folder_path = make_folder(pdf_path)

refs = refs_dict(pdf_path)

#test with one
cite, title = list(refs.items())[0]
refs = {cite:title}

get_docs(folder_path, refs)

locs = locs_dict(pdf_path, refs.keys())

ctx = ctx_dict(pdf_path, refs.keys())

sum = summaries_dict(folder_path, ctx)

The passage is discussing the performance of two attention mechanisms, additive attention and dot product attention, in terms of their scalability with respect to the dimensionality of the key vector (dk). The citation [3] refers to the paper that experimented with these attention mechanisms and found that additive attention outperforms dot product attention for larger values of dk.

In the cited paper, the authors compared the performance of additive attention (Equation 6) and multiplicative attention (Equation 7) and found that additive attention slightly but consistently outperformed multiplicative attention, regardless of the attention dimensionality (Table 5). This suggests that the additive attention mechanism is more robust to changes in the dimensionality of the key vector, which is the point being made in the original passage.
The citation [3] refers to the paper that introduced byte-pair encoding, a method for encoding sentences into a shared vocabulary of tokens. In the pass